In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda

# 노이즈 숫자 범위 설정
noise_min = 0.8  # 노이즈 최소값
noise_max = 1.2  # 노이즈 최대값

# 노이즈 숫자 생성 함수
def generate_noise(shape):
    return tf.random.uniform(shape=(shape,), minval=noise_min, maxval=noise_max)

# 모델 정의
def create_model():
    input_layer = Input(shape=(28, 28, 1))
    x = Conv2D(32, (3, 3), activation='relu')(input_layer)
    x = Lambda(lambda x: x + generate_noise(tf.shape(x)[-1]))(x)  # Conv 레이어 출력에 노이즈 추가
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = Lambda(lambda x: x + generate_noise(tf.shape(x)[-1]))(x)  # Conv 레이어 출력에 노이즈 추가
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = Flatten()(x)
    
    for layer in [Dense(64, activation='relu'), Dense(10, activation='softmax')]:
        output_with_noise = Lambda(lambda x: x + generate_noise(tf.shape(x)[-1]))(x)
        x = layer(output_with_noise)
    
    model = Model(inputs=input_layer, outputs=x)
    return model

# 모델 생성
model = create_model()

# 모델 컴파일
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 lambda_4 (Lambda)           (None, 26, 26, 32)        0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 lambda_5 (Lambda)           (None, 11, 11, 64)        0         
                                                           